## PARAMETRI DISTRIBUZIONE A PRIORI

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np

dip_intervals = []

for start in np.arange(0, 86, 5):
    end = start + 5
    interval = np.arange(start, end, 0.1)
    dip_intervals.append(interval)

In [4]:
# TABELLA 1

strike_intervals = [range(0, 45), range(45, 90), range(90, 135), range(135, 180), range(180, 225), range(225, 270), range(270, 315), range(315, 360 + 1)]

#dip_intervals = [np.arange(0, 22.5, 0.4), np.arange(22.5, 45, 0.5), np.arange(45, 67.5, 0.5), np.arange(67.5, 90 + 1, 0.5)]

rake_intervals = [range(-135, -45), range(-45, 45), range(45, 135), list(range(135, 180 + 1)) + list(range(-180, -135))]

In [5]:
# CREO UN DATAFRAME CHE CONTIENE TUTTE LE POSSIBILI COMBINAZIONI DEGLI INTERVALLI DELLA TABELLA 1

import pandas as pd
from itertools import product

combinations = list(product(strike_intervals, dip_intervals, rake_intervals))

# Trasforma le combinazioni in righe di un DataFrame
data = {
    
    'Strike Interval': [(c[0]) for c in combinations],
    
    'Dip Interval': [(c[1]) for c in combinations],
    
    'Rake Interval': [(c[2]) for c in combinations]

}

df = pd.DataFrame(data)

In [6]:
sigma = 20

def gaussian(x, preferred_dip, sigma):
    return np.exp(-(x - preferred_dip)**2 / (2 * sigma**2))

prob = []

for n in range(0, len(df)):
    
    if df['Rake Interval'][n] == range(-135, -45):  # FAGLIA NORMALE

        preferred_dip = 60
        
        x = np.linspace(df['Dip Interval'][n][0], df['Dip Interval'][n][1], 1000)
        
        y = gaussian(x, preferred_dip, sigma)

    elif df['Rake Interval'][n] == range(45, 135):  # FAGLIA INVERSA

        preferred_dip = 30
        
        x = np.linspace(df['Dip Interval'][n][0], df['Dip Interval'][n][1], 1000)
        
        y = gaussian(x, preferred_dip, sigma)

    elif df['Rake Interval'][n] == range(-45, 45):  # FAGLIA TRASCORRENTE

        preferred_dip = 90
        
        x = np.linspace(df['Dip Interval'][n][0], df['Dip Interval'][n][1], 1000)
        
        y = gaussian(x, preferred_dip, sigma)

    elif df['Rake Interval'][n] == list(range(135, 180 + 1)) + list(range(-180, -135)): ## FAGLIA TRASCORRENTE

        preferred_dip = 90
        
        x = np.linspace(df['Dip Interval'][n][0], df['Dip Interval'][n][1], 1000)
        
        y = gaussian(x, preferred_dip, sigma)
        
        
    prob.append(np.random.choice(y))
     

df['Probabilità'] = prob

In [7]:
n_prior = 100 # NUMERI DI DATI EQUIVALENTE

n_s = len(strike_intervals)

n_R = len(rake_intervals)

df['alpha'] = n_prior* (df['Probabilità'] / (n_s * n_R))

In [8]:
new_df = pd.DataFrame(columns=df.columns)

# Itera attraverso le righe del DataFrame originale
for index, row in df.iterrows():
    new_df = new_df.append(row)
    if (index + 1) % 4 == 0:
        new_df = new_df.append(row)

# Resetta gli indici del nuovo DataFrame
new_df.reset_index(drop=True, inplace=True)

In [9]:
# DEVO POI ACCORPARE I DUE INTERVALLI DELLE STRIKE SLIP!!!!


for index, row in new_df.iterrows():
    if (index + 1) % 4 == 0:  # Controlla se è ogni 3 righe
        new_df.at[index, 'Rake Interval'] = range(-180, -135)
        
for index, row in new_df.iterrows():
    if (index + 1) % 5 == 0:  # Controlla se è ogni 3 righe
        new_df.at[index, 'Rake Interval'] = range(135, 180)

In [10]:
#### GIRARE FIN QUA

## PARAMETRI LIKELIHOOD CMT 1

In [11]:

import pandas as pd

# CARICO IL CATALOGO CMT
df_CMT = pd.read_csv(r'CMT.csv')

In [12]:
def filtra_eventi(row):
    filtro = (
        (df_intervallo['Strike_min'] <= row['Strike1']) & (row['Strike1'] <= df_intervallo['Strike_max']) &
        (df_intervallo['Dip_min'] <= row['Dip1']) & (row['Dip1'] <= df_intervallo['Dip_max']) &
        (df_intervallo['Rake_min'] <= row['Rake1']) & (row['Rake1'] <= df_intervallo['Rake_max'])
    )
    return filtro.all()

#eventi_filtrati = df_CMT[df_CMT.apply(filtra_eventi, axis=1)]

In [13]:
# BISOGNA RICONTARE QUELLI STRIKE SLIP

In [14]:
conteggio = []
for n in range(0, len(new_df)):
    
    data_intervallo = {'Strike_min': [min(new_df['Strike Interval'][n])],
                       'Strike_max': [max(new_df['Strike Interval'][n])],
                       'Dip_min': [min(new_df['Dip Interval'][n])],
                       'Dip_max': [max(new_df['Dip Interval'][n])],
                       'Rake_min': [min(new_df['Rake Interval'][n])],
                       'Rake_max': [max(new_df['Rake Interval'][n])]}
    
    df_intervallo = pd.DataFrame(data_intervallo)
    
    eventi_filtrati = df_CMT[df_CMT.apply(filtra_eventi, axis=1)]
    
    conteggio.append(len(eventi_filtrati))
    
#conteggio

In [15]:
pd.DataFrame(conteggio).to_csv('conteggio.csv', index=True)

In [16]:
## caricare conteggio e aggiungerlo a new df

In [17]:
df_conteggio = pd.read_csv(r'conteggio.csv')

In [18]:
new_data = {'nuova_colonna': []}

for i in range(0, 720, 10):
    if i + 4 < 720:
        new_data['nuova_colonna'].extend([
            df_conteggio['0'][i],
            df_conteggio['0'][i + 1],
            df_conteggio['0'][i + 2],
            df_conteggio['0'][i + 3] + df_conteggio['0'][i + 4],
            df_conteggio['0'][i + 5],
            df_conteggio['0'][i + 6],
            df_conteggio['0'][i + 7],
            df_conteggio['0'][i + 8] + df_conteggio['0'][i + 9]
        ])
    else:
        new_data['nuova_colonna'].extend([df_conteggio['0'][j] for j in range(i, 720)])

new_data = pd.DataFrame(new_data)


In [19]:
df['Conteggio'] = new_data['nuova_colonna']

## PARAMETRI DISTRIBUZIONE A POSTERIORI CMT 1

In [20]:
# PARAMETRI 
df['alpha_posterior'] = df['alpha'] + df['Conteggio']

In [21]:
normal = df[df['Rake Interval'] == range(-135, -45)]

normal.reset_index(drop=True, inplace=True)

## CMT 1

In [22]:
conteggio_normal1 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = normal['Conteggio'][vettore].sum()
    
    conteggio_normal1.append(somma)


In [23]:
x = []

for n in  range (2, 92, 5):
    x.append(n + 0.5)

In [24]:
inverse = df[df['Rake Interval'] == range(45, 135)]

inverse.reset_index(drop=True, inplace=True)

In [25]:
conteggio_inverse1 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = inverse['Conteggio'][vettore].sum()
    
    conteggio_inverse1.append(somma)

In [26]:
ss = df[df['Rake Interval'] == range(-45, 45)]

ss.reset_index(drop=True, inplace=True)

In [27]:
conteggio_ss1 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = ss['Conteggio'][vettore].sum()
    
    conteggio_ss1.append(somma)

In [28]:
ss_2 = df.iloc[3::4]
ss_2.reset_index(drop=True, inplace=True)

In [29]:
conteggio_ss_21 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = ss_2['Conteggio'][vettore].sum()
    
    conteggio_ss_21.append(somma)

## CMT 2

In [30]:

import pandas as pd

# CARICO IL CATALOGO CMT
df_CMT = pd.read_csv(r'CMT.csv')

## LIKELIHOOD CMT 2

In [31]:
def filtra_eventi(row):
    filtro = (
        (df_intervallo['Strike_min'] <= row['Strike2']) & (row['Strike2'] <= df_intervallo['Strike_max']) &
        (df_intervallo['Dip_min'] <= row['Dip2']) & (row['Dip2'] <= df_intervallo['Dip_max']) &
        (df_intervallo['Rake_min'] <= row['Rake2']) & (row['Rake2'] <= df_intervallo['Rake_max'])
    )
    return filtro.all()

#eventi_filtrati = df_CMT[df_CMT.apply(filtra_eventi, axis=1)]

In [32]:
conteggio = []
for n in range(0, len(new_df)):
    
    data_intervallo = {'Strike_min': [min(new_df['Strike Interval'][n])],
                       'Strike_max': [max(new_df['Strike Interval'][n])],
                       'Dip_min': [min(new_df['Dip Interval'][n])],
                       'Dip_max': [max(new_df['Dip Interval'][n])],
                       'Rake_min': [min(new_df['Rake Interval'][n])],
                       'Rake_max': [max(new_df['Rake Interval'][n])]}
    
    df_intervallo = pd.DataFrame(data_intervallo)
    
    eventi_filtrati = df_CMT[df_CMT.apply(filtra_eventi, axis=1)]
    
    conteggio.append(len(eventi_filtrati))
    
#conteggio

In [33]:
pd.DataFrame(conteggio).to_csv('conteggio2.csv', index=True)

In [34]:
df_conteggio2 = pd.read_csv(r'conteggio2.csv')

In [35]:
new_data = {'nuova_colonna': []}

for i in range(0, 720, 10):
    if i + 4 < 720:
        new_data['nuova_colonna'].extend([
            df_conteggio2['0'][i],
            df_conteggio2['0'][i + 1],
            df_conteggio2['0'][i + 2],
            df_conteggio2['0'][i + 3] + df_conteggio2['0'][i + 4],
            df_conteggio2['0'][i + 5],
            df_conteggio2['0'][i + 6],
            df_conteggio2['0'][i + 7],
            df_conteggio2['0'][i + 8] + df_conteggio2['0'][i + 9]
        ])
    else:
        new_data['nuova_colonna'].extend([df_conteggio2['0'][j] for j in range(i, 720)])

new_data = pd.DataFrame(new_data)


In [36]:
df['Conteggio2'] = new_data['nuova_colonna']

## PARAMETRI POSTERIORI CMT 2

In [37]:
# PARAMETRI POSTERIORI CMT 2
df['alpha_posterior'] = df['alpha'] + df['Conteggio2']

In [38]:
normal = df[df['Rake Interval'] == range(-135, -45)]

normal.reset_index(drop=True, inplace=True)

In [39]:
conteggio_normal2 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = normal['Conteggio2'][vettore].sum()
    
    conteggio_normal2.append(somma)


In [40]:
inverse = df[df['Rake Interval'] == range(45, 135)]

inverse.reset_index(drop=True, inplace=True)

In [41]:
conteggio_inverse2 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = inverse['Conteggio2'][vettore].sum()
    
    conteggio_inverse2.append(somma)

In [42]:
ss = df[df['Rake Interval'] == range(-45, 45)]

ss.reset_index(drop=True, inplace=True)

In [43]:
conteggio_ss2 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = ss['Conteggio2'][vettore].sum()
    
    conteggio_ss2.append(somma)

In [44]:
ss_2 = df.iloc[3::4]
ss_2.reset_index(drop=True, inplace=True)

In [45]:
conteggio_ss_22 = []
for i in range(0, len(dip_intervals)):
    vettore = []
    numero = i

    # Generazione dei numeri nel vettore
    for _ in range(8):  # Puoi specificare quante volte vuoi aggiungere numeri al vettore
        vettore.append(numero)
        numero += len(dip_intervals)

    somma = ss_2['Conteggio2'][vettore].sum()
    
    conteggio_ss_22.append(somma)

In [50]:
import plotly.express as px

fig = px.bar(x = x, y = [conteggio_normal1, conteggio_normal2])#, labels={"wide_variable_0": "CMT1"})
             
fig.data[0].name = "CMT1"
fig.data[1].name = "CMT2"

fig.update_layout(title_text='Rake -135 -45')

fig.update_layout(
    xaxis=dict(
        title='Dip'#,
        #titlefont_size=16,
        #tickfont_size=14,
    ),
    #barmode='group',
    bargap=0, # gap between bars of adjacent location coordinates.
    #bargroupgap=0.1 # gap between bars of the same location coordinate.
)


fig.show()

In [47]:
fig = px.bar(x = x, y = [conteggio_inverse1, conteggio_inverse2])#, labels={"wide_variable_0": "CMT1"})
             
fig.data[0].name = "CMT1"
fig.data[1].name = "CMT2"

fig.update_layout(title_text='Rake 45 135')

fig.update_layout(
    xaxis=dict(
        title='Dip'#,
        #titlefont_size=16,
        #tickfont_size=14,
    ),
    #barmode='group',
    bargap=0, # gap between bars of adjacent location coordinates.
    #bargroupgap=0.1 # gap between bars of the same location coordinate.
)


fig.show()

In [48]:
fig = px.bar(x = x, y = [conteggio_ss1, conteggio_ss2])#, labels={"wide_variable_0": "CMT1"})
             
fig.data[0].name = "CMT1"
fig.data[1].name = "CMT2"

fig.update_layout(title_text='Rake -45 45')

fig.update_layout(
    xaxis=dict(
        title='Dip'#,
        #titlefont_size=16,
        #tickfont_size=14,
    ),
    #barmode='group',
    bargap=0, # gap between bars of adjacent location coordinates.
    #bargroupgap=0.1 # gap between bars of the same location coordinate.
)


fig.show()

In [49]:
fig = px.bar(x = x, y = [conteggio_ss_21, conteggio_ss_22])#, labels={"wide_variable_0": "CMT1"})
             
fig.data[0].name = "CMT1"
fig.data[1].name = "CMT2"

fig.update_layout(title_text='Rake 135:180 U -180:-135')

fig.update_layout(
    xaxis=dict(
        title='Dip'#,
        #titlefont_size=16,
        #tickfont_size=14,
    ),
    #barmode='group',
    bargap=0, # gap between bars of adjacent location coordinates.
    #bargroupgap=0.1 # gap between bars of the same location coordinate.
)


fig.show()